In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0,1,2,3

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub
import utils_bigbigan as ubigbi
print(tf.__version__)

In [ ]:
from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

In [ ]:
with tf.device('/device:GPU:0'):
    module = hub.Module('https://tfhub.dev/deepmind/bigbigan-resnet50/1') 
    gan_model = ubigbi.BigBiGAN(module)
    gen_ph = gan_model.make_generator_ph()
    gen_samples = gan_model.generate(gen_ph)
     
#     sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
#     init = tf.global_variables_initializer()
#     sess.run(init)

In [ ]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
from scipy.stats import truncnorm
import numpy as np
truncation = 1.0
std_scale = 1.0
batch_size = 4

seed = 0
state = None if seed is None else np.random.RandomState(seed)
zs = truncation * truncnorm.rvs(-2, 2, size=(batch_size, 120), random_state=state).astype(np.float32)
feed_dict = {gen_ph: zs}
anchor_out = sess.run(gen_samples, feed_dict=feed_dict)
